In [ ]:
!pip install -q -U watermark
!pip install -q spacy
!pip install -q transformers

In [ ]:
import math
import nltk
import spacy
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import transformers
from tokenizers import BertWordPieceTokenizer
from tqdm import tqdm
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from tensorflow import keras
from keras.utils import to_categorical
from tensorflow.keras import Sequential
from keras.preprocessing.text import Tokenizer
from keras.metrics import Precision, Recall, AUC
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.callbacks import EarlyStopping, LearningRateScheduler, CallbackList, ReduceLROnPlateau
from tensorflow.keras.optimizers.experimental import Adam

In [ ]:
%reload_ext watermark
%watermark -a "Marcelo Barreto"

# **Carregando os Dados de Texto**

In [ ]:
# Caso seja no Google Colab
'''from google.colab import drive

drive.mount('/content/drive')

import os

path = '/content/drive/MyDrive/Colab Notebooks' '''


In [ ]:
path = './datas'
# Carrega os dados de treino
train_data = pd.read_csv(os.path.join(path, 'dados_treinos.txt'), header = None, delimiter = ';')

# Carrega os dados de teste
test_data = pd.read_csv(os.path.join(path, 'dados_teste.txt'), header = None, delimiter = ';')

In [ ]:
# Ajusta os nomes das colunas
train_data = train_data.rename(columns = {0: 'texto', 1: 'sentimento'})
test_data = test_data.rename(columns = {0: 'texto', 1: 'sentimento'})

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
train_data.head()

In [ ]:
# Sentimentos presentes nos dados de treino
train_data['sentimento'].value_counts()

In [ ]:
# Sentimentos presentes nos dados de teste
test_data['sentimento'].value_counts()

# **Pré-processamento dos Dados de Texto com Spacy**

In [ ]:
# Baixando o dicionário
!python -m spacy download en_core_web_md

In [ ]:
# Carrega o dicionário
nlp_dict = spacy.load('en_core_web_md')

In [ ]:
# Definição da função 'preprocessar_texto' que recebe um texto como parâmetro
def preprocessar_texto(text):
  # Processa o texto usando o dicionário
  doc = nlp_dict(text)

  # Cria uma lista de lemas dos tokens, convertidos para minúsculas e sem espaços em brancos,
  # excluindo as palavras que são stopwords
  tokens = [token.lemma_.lower().strip() for token in doc if not token.is_stop]

  # Retorna os tokens processados como uma string única, unindo-os com espaços
  return ' '.join(tokens)

In [ ]:
# Aplicando a função nos dados de treino
train_data['texto_processado'] = train_data['texto'].apply(preprocessar_texto)

# Aplicando a função nos dados de teste
test_data['texto_processado'] = test_data['texto'].apply(preprocessar_texto)

In [ ]:
train_data.head()

# **Versão 1 do Modelo - Arquitetura Fully Connected Neural Network**

**Passo 1: Vetorização com TF-IDF**

In [ ]:
# Cria o Vetorizador
tfidf = TfidfVectorizer(max_df = 0.95, min_df = 2, stop_words = 'english')

In [ ]:
# Aplica o Vetorizador
tfidf_train_data = tfidf.fit_transform(train_data['texto_processado'])
tfidf_test_data = tfidf.transform(test_data['texto_processado'])

In [ ]:
tfidf_train_data.shape

In [ ]:
X_train_array = tfidf_train_data.toarray()
X_test_array = tfidf_test_data.toarray()

**Passo 2: Preparação dos Dados**

Converter variável alvo para representação numérica

In [ ]:
# Criar o Label Encoder
label_encoder = LabelEncoder()

# Aplica o Label Encoder
y_train_le = label_encoder.fit_transform(train_data['sentimento'])
y_test_le = label_encoder.transform(test_data['sentimento'])

In [ ]:
# Peso das Classes
w_classes = compute_class_weight('balanced', classes = np.unique(y_train_le), y = y_train_le)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_array,
                                                  y_train_le,
                                                  test_size = 0.2,
                                                  random_state = 42,
                                                  stratify = y_train_le)

In [ ]:
# Ajustar as variáveis para o tipo categórico
y_train_encoded = to_categorical(y_train)
y_val_encoded = to_categorical(y_val)
y_test_encoded = to_categorical(y_test_le)

In [ ]:
# Shape
y_train_encoded.shape, y_val_encoded.shape, y_test_encoded.shape

**Passo 3: Construção do Modelo**

In [ ]:
# Rede Neural Totalmente Conectada - Fully Connected Neural Network

model_v1 = Sequential()

# 1ª camada
model_v1.add(Dense(4096,
                   activation = 'selu', # função de ativação SELU (Scaled Exponential Linear Unit)
                   kernel_initializer = 'lecun_normal', # inicializa os pesso com a distribuição lecun normal
                   input_shape = (X_train.shape[1],),
                   kernel_regularizer = tf.keras.regularizers.l2(0.01))) # Aplica regularização l2 para evitar overfitting

# 2ª camada
model_v1.add(Dense(2048,
                   activation = 'selu',
                   kernel_initializer = 'lecun_normal',
                   kernel_regularizer = tf.keras.regularizers.l2(0.01)))

# 3ª camada
model_v1.add(Dense(1024,
                   activation = 'selu',
                   kernel_initializer = 'lecun_normal',
                   kernel_regularizer = tf.keras.regularizers.l2(0.1)))

# 4ª camada
model_v1.add(Dense(64, activation = 'selu'))

# 5ª camada - camada de saída
model_v1.add(Dense(6, activation = 'softmax'))

**Passo 4: Compilação e Sumário do Modelo**

In [ ]:
# Atribui pesos específicos ao vetor de bias da última camada do modelo
model_v1.layers[-1].bias.assign(w_classes)

In [ ]:
# Compila o modelo
# Define o otimizador como 'Adam'
# Adam é um algoritmo de otimização que pode ser usado no lugar do procedimento clássico da descida
# do gradiente estocástica para atualizar os pesos da rede iterativamente com base nos dados de treinamento,
# Define a função de perda como 'categorical_crossentropy'. É adequada para problemas de classificação
# multiclasse, onde os rótulos são fornecidos em um formato one-hot encoded.
# Define a métrica de avaliação do modelo como 'accuracy' (acurácia).
# A acurácia é uma métrica comum para avaliar o desempenho de modelos de classificação
model_v1.compile(optimizer = 'Adam',
                 loss = tf.losses.categorical_crossentropy,
                 metrics = ['accuracy', Precision(), Recall(), AUC()])

In [ ]:
model_v1.summary()

**Passo 5: Callbacks e Early Stopping**

In [ ]:
# Função para o Parâmetro de Scheduler de Taxa de Aprendizado
def step_decay(epoch):
    initial_rate = 0.001
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_rate * math.pow(drop, math.floor((1 + epoch) / epochs_drop))
    return lrate

In [ ]:
# Scheduler de Taxa de Aprendizado
lr_scheduler = LearningRateScheduler(step_decay)

In [ ]:
# Early Stopping
early_stopping = EarlyStopping(monitor = 'val_loss', restore_best_weights = True, patience = 3)

In [ ]:
# Hiperparâmetros
num_epochs = 20
tamanho_batch = 256

In [ ]:
%%time
history = model_v1.fit(X_train,
                       y_train_encoded,
                       validation_data = (X_val, y_val_encoded),
                       epochs = num_epochs,
                       batch_size = tamanho_batch,
                       callbacks = [early_stopping, lr_scheduler])

**Passo 7: Avaliação do Modelo**

In [ ]:
# Extrai o erro em treino e validação
loss, val_loss = history.history['loss'], history.history['val_loss']

In [ ]:
# Gráfico do Valor de Perda do Modelo
plt.plot(loss, label = 'loss')
plt.plot(val_loss, label = 'val_loss')
plt.legend()
plt.show()

In [ ]:
# Previsões com dado de teste
predict_v1 = model_v1.predict(X_test_array)

In [ ]:
# Extrai as labels
predict_v1_labels = predict_v1.argmax(axis = 1)

In [ ]:
print(classification_report(y_test_le, predict_v1_labels))

In [ ]:
print(confusion_matrix(y_test_le, predict_v1_labels))

In [ ]:
print(accuracy_score(y_test_le, predict_v1_labels))

In [ ]:
model_v1.save('./datas/model_v1.keras')

**Passo 8: Deploy da Versão 1 do Modelo**

In [ ]:
# Carrega o modelo
loaded_model = keras.models.load_model('./datas/model_v1.keras')

In [ ]:
# Nova frase
phrase = "i even feel a little shaky"

In [ ]:
# Cria um DataFrame da frase
df = pd.DataFrame({'Phrase': [phrase]})

In [ ]:
# Aplica a função de PreProcessamento
df['Processed_Phrase'] = df['Phrase'].apply(preprocessar_texto)

In [ ]:
df

In [ ]:
# Aplica a vetorização
df_tfidf = tfidf.transform(df['Processed_Phrase'])

In [ ]:
# Transforma em array
df_tfidf_array = df_tfidf.toarray()

In [ ]:
# Previsões
results = loaded_model.predict(df_tfidf_array)

In [ ]:
prob_class = np.argmax(results, axis = 1)
prob_class

In [ ]:
# Obtém o nome da classe
class_name = label_encoder.inverse_transform(prob_class)
class_name